クロス集計とカイ二乗検定

- ``core.py``のモジュールに、クロス集計＆カイ二乗検定するメソッドを追加する
  - ``crosstab_data`` : クロス集計した結果をカイ二乗検定する
  - ``crosstab_heatmap`` : クロス集計した結果をヒートマップにする
  - ``crosstab``: 上記のメソッドをまとめて実行する

In [1]:
import itertools
import altair as alt
import pandas as pd
import titanite as ti
import scipy as sp

print(f"Altair {alt.__version__}")
print(f"Pandas {pd.__version__}")
print(f"SciPy {sp.__version__}")
print(f"Titanite {ti.__version__}")

Altair 5.0.1
Pandas 2.0.3
SciPy 1.11.1
Titanite 0.4.0


データを読み込む

- 前処理済のデータなので``Data``クラスで読み込む
  - データ読み込みをシンプルにできた
  - 外から``Config``にアクセスできないデメリットがある

In [2]:
f_cfg = "../sandbox/config.toml"
f_csv = "../data/test_data/prepared_data.csv"
d = ti.Data(read_from=f_csv, load_from=f_cfg)
data = d.read()
# data.info()

2023-07-23 14:19:27.800 | INFO     | titanite.preprocess:categorical_data:123 - Categorize
2023-07-23 14:19:27.807 | INFO     | titanite.preprocess:binned_data:229 - Binned


クロス集計すると、データのカテゴリ情報がなくなってしまうので、対処法を調べてみた

- ``pd.DataFrame.dtypes``ですべてのカラムの型を確認できる
- ``pd.Series.dtypes`` / ``pd.Series.dtype``でカラムの型を確認できる
- ``data.cat.categories``でカテゴリーの中身にアクセスできる
- 型の変換なので``pd.DataFrame.astype``でキャストすればよい
- ``astype``には``pd.Series.dtype``を渡せばよいことがわかった

In [3]:
#data.dtypes

In [4]:
#data["q01"].cat.categories

In [5]:
# for h in data.columns:
#     t = data[h].dtype
#     if t == "category":
#         c = data[h].cat.categories
#         print(f"{h} = {c}")

クロス集計してカイ二乗検定する

In [6]:
x = "q02"
y = "q03"
v = "count"

# data[h0].dtype
# data[h1].dtype

In [7]:
cross_tab = pd.crosstab(data[x], data[y])
#cross_tab
#cross_tab.columns
#cross_tab.info()

クロス集計した結果をロングデータに変換する（グラフ作成のための準備）
- ``reset_index``するときに、カラムの型情報が抜けてしまった
- カラム名が同じなので、元データのカテゴリ型を使って、上書きする

In [8]:
melted = cross_tab.reset_index().melt(
    id_vars=x,
    var_name=y,
    value_name=v,
    )
#ti.categorical_data(melted, category)
melted[x] = melted[x].astype(data[x].dtype)
melted[y] = melted[y].astype(data[y].dtype)
#melted.info()

これまでの処理をまとめる

In [29]:
from scipy.stats import chi2_contingency

def crosstab(data: pd.DataFrame, header: tuple):
    x = header[0]
    y = header[1]
    z = "count"

    # クロス集計してカイ二乗検定
    cross_tab = pd.crosstab(data[x], data[y])
    chi2_test = chi2_contingency(cross_tab)

    # ロングデータに変換
    melted = cross_tab.reset_index().melt(id_vars=x, var_name=y, value_name=z)
    # 元データのカテゴリ型情報を付け直す
    melted[x] = melted[x].astype(data[x].dtype)
    melted[y] = melted[y].astype(data[y].dtype)

    # グラフを作成
    base = alt.Chart(melted).encode(
        alt.X(x),
        alt.Y(y),
    )

    mark = base.mark_rect().encode(
        color = alt.condition(
        alt.datum[z] > 0,
        alt.Color(f"{z}:Q"),
        alt.value("white"),
        )
    )

    text = base.mark_text().encode(
        text=alt.condition(
        alt.datum[z] > 0,
        alt.Text(f"{z}:Q"),
        alt.value("")
    ),
    color=alt.value('black')
    )

    heatmap = (mark + text).properties(
        width=400,
        height=400,
    )
    return cross_tab, chi2_test, heatmap


In [30]:
cross_tab, chi2_test, hm = crosstab(data, ("q01", "q02"))

In [31]:
# cross_tab.info()
cross_tab

q02,Male,Female,Non-binary,Prefer to self-identify,Prefer not to answer
q01,,,,,
20s,67,32,4,0,0
30s,51,33,1,2,0
40s,26,11,0,1,1
50s,22,5,0,0,0
60s,16,3,0,0,1
70s,5,0,0,0,0
80s,2,0,0,0,0
90s+,2,1,0,0,0
Prefer not to answer,0,1,0,0,0


In [32]:
hm

alt.LayerChart(...)

In [13]:
print(chi2_test.statistic)
print(chi2_test.pvalue)
print(chi2_test.dof)

29.775319946037516
0.5795956041049015
32


カイ二乗検定の結果をCSV（or JSON）に書き出す

- まずデータフレームに変換する
- JSONはTypstで読み込みやすい
- CSVはTypstでそのまま表にできそう

In [14]:
headers = [
    ("q01", "q02"),
    ("q13_binned", "q14"),
    ("q03_subregional", "q19"),
]

In [15]:
chi2_tests = []
for header in headers:
    cross_tab, chi2_test, hm = crosstab(data, header)
    #display(cross_tab)
    #hm.display()
    x, y = header
    name = f"{x}-{y}"
    print(name)
    print(chi2_test.statistic)
    print(chi2_test.pvalue)
    print(chi2_test.dof)
    r = [name, chi2_test.statistic, chi2_test.pvalue, chi2_test.dof]
    chi2_tests.append(r)

q01-q02
29.775319946037516
0.5795956041049015
32
q13_binned-q14
377.3278332834723
1.8953886707793875e-38
85
q03_subregional-q19
136.9978979904653
7.629276389468528e-05
80


In [16]:
chi2_tests

[['q01-q02', 29.775319946037516, 0.5795956041049015, 32],
 ['q13_binned-q14', 377.3278332834723, 1.8953886707793875e-38, 85],
 ['q03_subregional-q19', 136.9978979904653, 7.629276389468528e-05, 80]]

In [17]:
pd.DataFrame(chi2_tests, columns=["name", "statistic", "p-value", "dof"])

,name,statistic,p-value,dof
0,q01-q02,29.775320,5.795956e-01,32
1,q13_binned-q14,377.327833,1.895389e-38,85
2,q03_subregional-q19,136.997898,7.629276e-05,80
